# Training and Testing

In this section we put together everything we learned about images and features so that we can train a machine learning algorithm to distinguish between the images of different tags.